# Example analysis: Most significantly effective intervention for every outcome
### This does not take into account any variables except _intervention\_id_ and outcome

In [ ]:
# Import N-of-1 analysis library by Thomas Gaertner: Analyse causal inferences in N-of-1 studies https://github.com/thogaertner/cinof1
library(cinof1)

raw_df = read.csv(file = 'data/subjects.csv')

significantly_effective_intervention <- function(raw_df, outcome, exposure='intervention_id') {
    df <- data.frame(raw_df)
    df[,exposure] = factor(raw_df[,exposure])
    drop <- c('completed_at', 'result_type', 'user_id', 'study_id', 'started_at', 'completed_at', 'invite_code')
    df = df[,!(names(df) %in% drop)]

    # Shapiro–Wilk to test for normal distribution
    shapiro_result = shapiro.test(df[,outcome])
    print(shapiro_result)
    is_normal_distribution = shapiro_result$p >= 0.05 # Check if null hypothesis (nomal distribution) is likely
    if (is_normal_distribution) cat("The Shapiro–Wilk test: NORMAL distribution\n\n\n") else cat("The Shapiro–Wilk test: NO normal distribution\n\n\n")

    performance_difference <- function(data) {
        if (is_normal_distribution) {
            cat("Normal distribution found. Applying Welch Two Sample t-test")
            return(t.test(as.formula(sprintf("%s ~ %s", outcome, exposure)), data = data, exact = FALSE, correct = FALSE, conf.int = FALSE))
        } else {
            cat("No normal distribution found. App%slying Wilcoxon signed rank test")
            # Using cinof1 package
            return(wilcox.nofone(data, exposure = exposure, outcome = outcome))
        }
    }
    diff = performance_difference(df)
    print(diff)
    cat("\n\n")
    has_significant_difference = diff$p.value < 0.05

    if (has_significant_difference) {
        means = aggregate(as.formula(sprintf("%s ~ %s", outcome, exposure)), data=df, mean)
        if (means[,outcome][1] > means[,outcome][2]) {
            cat('The intervention "', levels(means[,exposure])[1],'" is more effective by a mean of ', means[,outcome][1] - means[,outcome][2], ' to increase "', outcome, '" outcome')
        } else {
            cat('The intervention "', levels(means[,exposure])[2], '" is more effective by a mean of ', means[,outcome][2] - means[,outcome][1], ' to increase "', outcome, '" outcome')
        }
    } else {
        cat("Difference between means of interventions is NOT significant: ", diff$p.value)
    }
    cat("\n\n")
    
    # Using cinof1 package
    print(comparative.plot(raw_df, exposure = exposure, outcome = outcome))
}

## Most significantly effective intervention for 'pain'

In [ ]:
significantly_effective_intervention(outcome = 'pain')